In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

pi = np.pi
exp = np.exp

In [ ]:
Nq = 2  # number of levels in qubit
N1 = 3  # number of levels in resonator 1
N2 = 3  # number of levels in resonator 2

wq = 7e9*2*pi  # qubit frequency
w1 = wq        # resonator 1 frequency
w2 = wq        # resonator 2 frequency
wp = wp        # pump frequency

kq = 50e3*2*pi
k1 = 5e3*2*pi
k2 = 6e6*2*pi

g1 = 1e6*2*pi  # drive frequency resonator 1
g2 = 5e6*2*pi  # drive frequency resonator 2
Omega = 1e6*2*pi  # pump frequency

Ec = 200e6*2*pi  # Anharmonicity

t0 = 0            # start Gaussian pulse Omega
t1 = t0 + 50e-9   # end Gaussian pulse Omega
t2 = t1 + 10e-9   # start square pulse at resonator 1
t3 = t2 + 300e-9  # end square pulse at resonator 1
t4 = t3 + 10e-9   # start square pulse at resonator 2
t5 = t4 + 300e-9  # end square pulse at resonator 2
t6 = t5 + 10e-9   # t0
tg = 10e-9        # Gaussian rise and fall time of square pulse

N_repeat = 2  # number of times to repeat cycle

In [ ]:
# Transmon operators
b = tensor(destroy(Nq), qeye(N1), qeye(N2))
n = b.dag()*b

# Cavity operators
a1 = tensor(qeye(Nq), destroy(N1), qeye(N2))
a2 = tensor(qeye(Nq), qeye(N1), destroy(N2))

# Jaynes-Cummings Hamiltonian
Hjc = ( (w1*a1.dag()*a1) 
      + (w2*a2.dag()*a2) 
      + (wq*b.dag()*b) 
      + Ec/2*b.dag()*b.dag()*b*b )

# Coupling Hamiltonians
H1 = g1*(a1*b.dag() + b*a1.dag())
H2 = g2*(a1*b.dag() + b*a2.dag())

# Pump Hamiltonian
Hp = Omega/2*(b + b.dag)  # TODO: add rotating frame